In [1]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.forms.models import model_to_dict

from scoping.models import *
from tmv_app.models import *
from scipy import stats
import ipy_table as tbl
from itertools import islice

qid = 1498

In [20]:
df = pd.read_csv('bib_data/NETS [Nodes].csv')

cols = df.columns

df.head()



,id,label,timeset,weight<total_link_strength>,score<citations>,score<pub._year>,weight<links>,weight<citations>,cluster,description,url
0,1,carter (2017),NaN,280,0,2017,175,0,4,"<table><tr><td>Authors:</td><td>carter, ea; se...",http://dx.doi.org/10.1021/acs.accounts.6b00479
1,2,jones (2017),NaN,411,0,2017,93,0,4,"<table><tr><td>Authors:</td><td>jones, cw; lee...",http://dx.doi.org/10.1021/jacs.7b00235
2,3,guo (2017),NaN,96,0,2017,80,0,4,"<table><tr><td>Authors:</td><td>guo, fs; wang,...",http://dx.doi.org/10.1039/c6dt04751c
3,4,dai (2017),NaN,419,1,2017,110,1,4,"<table><tr><td>Authors:</td><td>dai, s; huang,...",http://dx.doi.org/10.1016/j.cej.2016.12.004
4,5,custelcean (2017),NaN,563,0,2017,171,0,4,"<table><tr><td>Authors:</td><td>custelcean, r;...",http://dx.doi.org/10.1002/anie.201610916


In [28]:
p = Project.objects.get(title="NETs")
techs = Technology.objects.filter(project = p).values_list('name',flat=True)

ndf = df

def istech(x,t):
    doi = str(x['url']).replace('http://dx.doi.org/','').strip()

    try:
        doc = Doc.objects.get(
                wosarticle__di=doi
            )
    except:
        print(doi)
        return(np.nan)

    if doc.technology.name==t:
        return(1)
    elif DocOwnership.objects.filter(
            doc=doc,
            query__technology__name=t,
            relevant=1
        ).count() > 0:
        return(1)
    else:
        return(0)

for t in techs:
    ndf[t] = ndf.apply(lambda x: istech(x,t),axis=1)
    
ndf.head()

10.1016/s2095-3119(16)61337-0
10.1108/ijccsm-09-2015-0135
nan
10.4067/s0718-221x2016005000053
nan
10.1002/ente.201600186
10.1017/s0376892916000199
10.1002/ldr.2310
10.1080/1943815x.2016.1159578
nan
10.1175/jamc-d-16-0135.1
10.1002/2016gl068576
10.1016/j.eneco.2015.04.011
10.1016/j.eneco.2016.01.005
10.1016/j.eneco.2015.11.019
nan
10.1111/2041-210x.12505
10.1071/sr14343
10.1038/nclimate2882
10.1093/aepp/ppv010
10.3197/096327115x14497392134964
10.3197/096327115x14497392134766
10.1016/j.techfore.2015.06.001
10.1038/nclimate2729
nan
10.1071/rj15070
nan
10.1016/s2095-3119(15)61068-1
10.3103/s106837391507002x
10.1071/rj14109
nan
10.1002/2014ef000249
10.1016/j.fuproc.2015.03.022
10.1130/g36449.1
nan
nan
nan
10.1002/2014ms000361
10.1016/s0306-2619(97)00020-2
10.1023/a:1026494514131
nan
10.1007/bf00142580
nan
10.1007/bf01204199
10.1007/bf01204198
10.1007/bf00546765
nan
10.1016/0961-9534(95)00044-5
nan
10.1016/0360-5442(95)00035-f
10.1007/bf01091928
10.1029/94gb00993
10.1007/bf01098474
nan
10.10

,id,label,timeset,weight<total_link_strength>,score<citations>,score<pub._year>,weight<links>,weight<citations>,cluster,description,...,BECCS,Soil Carbon Sequestration,Ocean Alkalinisation,Bioenergy,Afforestation/reforestation,Ethics & Morals,Biochar,Direct Air Capture,Ocean fertilisation,Synonyms
0,1,carter (2017),NaN,280,0,2017,175,0,4,"<table><tr><td>Authors:</td><td>carter, ea; se...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2,jones (2017),NaN,411,0,2017,93,0,4,"<table><tr><td>Authors:</td><td>jones, cw; lee...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,guo (2017),NaN,96,0,2017,80,0,4,"<table><tr><td>Authors:</td><td>guo, fs; wang,...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,dai (2017),NaN,419,1,2017,110,1,4,"<table><tr><td>Authors:</td><td>dai, s; huang,...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,custelcean (2017),NaN,563,0,2017,171,0,4,"<table><tr><td>Authors:</td><td>custelcean, r;...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [29]:
ndf.head()

len(ndf)

len(ndf[ndf['BECCS'].isnull()])

249

In [30]:
melted_ndf = pd.melt(ndf,id_vars=list(cols))
melted_ndf.head()

,id,label,timeset,weight<total_link_strength>,score<citations>,score<pub._year>,weight<links>,weight<citations>,cluster,description,url,variable,value
0,1,carter (2017),NaN,280,0,2017,175,0,4,"<table><tr><td>Authors:</td><td>carter, ea; se...",http://dx.doi.org/10.1021/acs.accounts.6b00479,Enhanced Weathering,0.0
1,2,jones (2017),NaN,411,0,2017,93,0,4,"<table><tr><td>Authors:</td><td>jones, cw; lee...",http://dx.doi.org/10.1021/jacs.7b00235,Enhanced Weathering,0.0
2,3,guo (2017),NaN,96,0,2017,80,0,4,"<table><tr><td>Authors:</td><td>guo, fs; wang,...",http://dx.doi.org/10.1039/c6dt04751c,Enhanced Weathering,0.0
3,4,dai (2017),NaN,419,1,2017,110,1,4,"<table><tr><td>Authors:</td><td>dai, s; huang,...",http://dx.doi.org/10.1016/j.cej.2016.12.004,Enhanced Weathering,0.0
4,5,custelcean (2017),NaN,563,0,2017,171,0,4,"<table><tr><td>Authors:</td><td>custelcean, r;...",http://dx.doi.org/10.1002/anie.201610916,Enhanced Weathering,0.0


In [76]:
ct_sum = melted_ndf.groupby(['cluster', 'variable']).agg({
    'value': 'sum'
})

ct_sum = ct_sum.reset_index().sort_values('cluster')

ct_totals = ct_sum.groupby(['cluster']).sum().rename(columns={'value':'total'}).reset_index()

ct_sum = ct_sum.merge(ct_totals)

ct_sum['prop'] = ct_sum['value'] / ct_sum['total'] * 100

ct_sum = ct_sum.sort_values(['cluster','prop'], ascending=[True, False])

ct_sum.to_excel('bib_data/cluster_totals.xlsx')

ct_sum.head(15)


,cluster,variable,value,total,prop
0,1,Afforestation/reforestation,215.0,265.0,81.132075
2,1,Soil Carbon Sequestration,44.0,265.0,16.603774
10,1,BECCS,3.0,265.0,1.132075
1,1,Synonyms,1.0,265.0,0.377358
7,1,Blue Carbon,1.0,265.0,0.377358
9,1,Biochar,1.0,265.0,0.377358
3,1,Ocean fertilisation,0.0,265.0,0.000000
4,1,Ethics & Morals,0.0,265.0,0.000000
5,1,Enhanced Weathering,0.0,265.0,0.000000
6,1,Ocean Alkalinisation,0.0,265.0,0.000000
